In [ ]:
import konlpy
from konlpy.tag import Okt
import json
import os
import nltk
import numpy as np

# [Lab] 학습 모델 로딩 및 테스트


## 학습을 위한 라이브러리 읽기

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

## 데이터 로딩 및 준비

In [ ]:
# 데이터 로딩



# 전체 단어 개수 파악
tokens = [t for d in train_docs for t in d[0]]

okt = Okt()
def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

# NLTK의 Text 클래스는 문서 분석에 유용한 여러가지 메서드를 제공
text = nltk.Text(tokens, name='NMSC') 

selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

## 모델 로딩

In [ ]:
# load json and create model
 
    
json_file.close()
loaded_model = models.model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

## 문장 입력 테스트

In [ ]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(loaded_model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 \"긍정\" 추측\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 \"부정\" 추측\n".format(review, (1 - score) * 100))

## 테스트 문장 입력

In [ ]:
predict_pos_neg("올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.")
predict_pos_neg("배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.")
predict_pos_neg("주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ")
predict_pos_neg("믿고 보는 감독이지만 이번에는 아니네요")
predict_pos_neg("주연배우 때문에 봤어요")

In [ ]:
predict_pos_neg("굳이 미라랑 키스를 왜 하는지.")
predict_pos_neg("진짜 최고최고최고 재개봉 덕분에 오늘 4D로 보고왔는데 진짜 개돌았음.")
predict_pos_neg("거리에 살고 있지만 우리의 이웃...당연히 사랑스러운 존재^^")
predict_pos_neg("바다판 영화 '라이프'. 코스믹호러를 통해 위압감과 공포심을 주려고 하는 것 같지만 그런 묘사가 너무 적고 영화의 마지막까지 무엇과 싸우는지 확실하게 인식이 불가능. 결말은 그냥 해당 사건을 무마시켰다는게 결말. 차라리 라이프를 보는게 100배는 재밌습니다")
predict_pos_neg("개 쓰레기 영화 첨으로 리뷰단다...와이런것도 영화라고 허허ㅋㅋ")

In [ ]:
predict_pos_neg("남자 주인공이 멋있다")

# 참고용 학습 코드 

## 학습, 테스트 데이터 준비

In [ ]:
#데이터 파일 읽는 함수 정의
def read_data(filename):
    with open(filename, 'r', encoding='UTF-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # txt 파일의 헤더(id document label)는 제외하기
    return data

#데이터 파일 읽기
train_data = read_data('data/ratings_train.txt')
test_data = read_data('data/ratings_test.txt')

In [ ]:
# 데이터 개수 확인
 
    

In [ ]:
# DataFrame 생성 및 변환
#import pandas as pd
#df = pd.DataFrame(test_data, columns=['id','text','rating'])
#df[:200]

# 데이터 전처리

## 형태소 분석

In [ ]:
# 토크나이징 및 형태소 분석 함수 정의
okt = Okt()

def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

# 토크나이즈 및 형태소 분석 수행 및 Json 파일로 저장
if os.path.isfile('data/train_docs.json'): # 이미 관련 파일이 있으면 파일 읽기
    with open('data/train_docs.json', encoding='UTF-8') as f:
        train_docs = json.load(f)
    with open('data/test_docs.json', encoding='UTF-8') as f:
        test_docs = json.load(f)
else: # 파일이 없으면 Json 파일 생성 저장
    # 형태소 분석 수행
    train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
    test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
    # JSON 파일로 저장
    with open('data/train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('data/test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

In [ ]:
# 분석 결과 일부 출력
print(test_docs[0])
print(test_docs[1])
print(test_docs[2])
print(test_docs[3])
print(test_docs[4])

In [ ]:
# 전체 단어 개수 파악
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

# NLTK의 Text 클래스는 문서 분석에 유용한 여러가지 메서드를 제공
text = nltk.Text(tokens, name='NMSC') 

# 중복을 제외한 토큰의 개수 출력
print(len(set(text.tokens)))  

# 출현 빈도가 높은 상위 토큰 10개 출력
print(text.vocab().most_common(10))

## 토큰 개수 파악 - 저빈도 단어 제거 및 벡터 생성

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
%matplotlib inline

font_fname = 'c:/windows/fonts/gulim.ttc' # 윈도우용
#font_fname = '/System/Library/Fonts/Supplemental/AppleGothic.ttf' # 맥용
font_name = font_manager.FontProperties(fname=font_fname).get_name()
rc('font', family=font_name)

# 그래프 그리기 

    

## [여기부터 장시간 소요]]] 사용되는 토큰 10000개를 사용하여 벡터 생성


In [ ]:
selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

## 학습(deep learning)을 위해 데이터를 float 형태로 변환

In [ ]:
import numpy as np

# 데이터 변환
x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')
y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

# 데이터 저장
np.savetxt("data/x_test.txt", x_test, fmt='%f', delimiter=',')
np.savetxt("data/x_train.txt", x_train, fmt='%f', delimiter=',')
np.savetxt("data/y_train.txt", y_train, fmt='%f', delimiter=',')
np.savetxt("data/y_test.txt", y_test, fmt='%f', delimiter=',')

# Deep learning 활용 학습

## float 형태로 변형된 데이터 읽기

In [ ]:
import numpy as np

x_test = np.loadtxt("data/x_test.txt", dtype=float, delimiter=',')
x_train = np.loadtxt("data/x_train.txt", dtype=float, delimiter=',')
y_train = np.loadtxt("data/y_train.txt", dtype=float, delimiter=',')
y_test = np.loadtxt("data/y_test.txt", dtype=float, delimiter=',')

## 학습 모델 생성 및 학습

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)
results

In [ ]:
# 모델 저장
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

## 테스트 데이터를 통한 정확도 테스트

In [ ]:
# 테스트 데이터 (약 4.5GB) 로딩
#x_test = np.loadtxt("data/x_test.txt", dtype=float, delimiter=',')
#y_test = np.loadtxt("data/y_test.txt", dtype=float, delimiter=',')

# 정확도 측정
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

## 문장 입력 테스트

In [ ]:
def predict_pos_neg(review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    score = float(model.predict(data))
    if(score > 0.5):
        print("[{}]는 {:.2f}% 확률로 긍정 리뷰 추측\n".format(review, score * 100))
    else:
        print("[{}]는 {:.2f}% 확률로 부정 리뷰 추측\n".format(review, (1 - score) * 100))

In [ ]:
predict_pos_neg("올해 최고의 영화! 세 번 넘게 봐도 질리지가 않네요.")
predict_pos_neg("배경 음악이 영화의 분위기랑 너무 안 맞았습니다. 몰입에 방해가 됩니다.")
predict_pos_neg("주연 배우가 신인인데 연기를 진짜 잘 하네요. 몰입감 ㅎㄷㄷ")
predict_pos_neg("믿고 보는 감독이지만 이번에는 아니네요")
predict_pos_neg("주연배우 때문에 봤어요")

In [ ]:
predict_pos_neg("^^")

In [ ]:
predict_pos_neg("굳이 미라랑 키스를 왜 하는지.")
predict_pos_neg("진짜 최고최고최고 재개봉 덕분에 오늘 4D로 보고왔는데 진짜 개돌았음.")
predict_pos_neg("거리에 살고 있지만 우리의 이웃...당연히 사랑스러운 존재^^")
predict_pos_neg("바다판 영화 '라이프'. 코스믹호러를 통해 위압감과 공포심을 주려고 하는 것 같지만 그런 묘사가 너무 적고 영화의 마지막까지 무엇과 싸우는지 확실하게 인식이 불가능. 결말은 그냥 해당 사건을 무마시켰다는게 결말. 차라리 라이프를 보는게 100배는 재밌습니다")
predict_pos_neg("개 쓰레기 영화 첨으로 리뷰단다...와이런것도 영화라고 허허ㅋㅋ")